In [1]:
import tensorflow as tf

In [4]:
a = tf.constant([1, 2, 3])
b = tf.constant([1, 2, 3])

c = a + b

In [5]:
with tf.Session() as session:
    print(c.eval())

[2 4 6]


In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14616975514637075866
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6690037432
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16514736616002063321
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


## Heron's Formula in TensorFlow

In [7]:
def compute_area(sides):
    a = sides[:,0]
    b = sides[:,1]
    c = sides[:,2]
    
    s = (a + b + c) * 0.5
    areasq = s * (s - a) * (s - b) * (s - c)
    return tf.sqrt(areasq)

with tf.Session() as session:
    area = compute_area(tf.constant([
        [5.0, 3.0, 7.1],
        [2.3, 4.1, 4.8]
    ]))
    print(session.run(area))

[6.278497 4.709139]


## Halley's Method

In [2]:
def get_constants(constants):
    return constants[0], constants[1], constants[2], constants[3], constants[4]

In [3]:
def fx(constants, x):
    a0, a1, a2, a3, a4 = get_constants(constants)
    return a0 + (a1 * x) + (a2 * tf.pow(x, 2)) + (a3 * tf.pow(x, 3)) + (a4 * tf.pow(x, 4))

In [4]:
def f1x(constants, x):
    a0, a1, a2, a3, a4 = get_constants(constants)
    return a1 + (2 * a2 * x) + (3 * a3 * tf.pow(x, 2)) + (4 * a4 * tf.pow(x, 3))

In [5]:
def f2x(constants, x):
    a0, a1, a2, a3, a4 = get_constants(constants)
    return (2 * a2) + (6 * a3 * x) + (12 * a4 * tf.pow(x, 2))

In [6]:
def halley_iter(x, _fx, _f1x, _f2x):
    return x - ((2 * _fx * _f1x) / ((2 * tf.pow(_f1x, 2)) - (_fx * _f2x)))

In [44]:
def body(nx, ox, constants, epi):
    ox = nx
    nx = halley_iter(ox, fx(constants, ox), f1x(constants, ox), f2x(constants, ox))
    return [nx, ox, constants, epi]

def condition(nx, ox, constants, epi):
    return tf.greater_equal(tf.reshape(tf.abs(nx - ox), []), epi)

def halley(constants, x, epi):
    with tf.variable_scope('model', reuse=tf.AUTO_REUSE):
        ox = tf.get_variable(name='old-x', initializer=tf.constant_initializer(0.0), trainable=True)
        nx = tf.get_variable(name='new-x', initializer=tf.constant_initializer(x), trainable=True)
    a, b, c, d = tf.while_loop(condition, body, [nx, ox, constants, epi])
    print('x = {}'.format(a.eval()))
    print('f(x) = {}'.format(fx(constants, a).eval()))
    

In [45]:
with tf.Session() as session:
    x = 1.0
    constants = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0])
    epi = tf.constant(0.001)
    tf.global_variables_initializer().run()
    halley(constants, x, epi)

x = [[-0.4358876]]
f(x) = [[0.54744387]]
